In [1]:
!pip install python-pptx sentence-transformers faiss-cpu pandas tqdm


In [11]:
import sys
!{sys.executable} -m pip install pymupdf sentence-transformers faiss-cpu pandas tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 6.5 MB/s eta 0:00:0000:0100:01


In [2]:
import sys
!{sys.executable} -m pip install python-pptx sentence-transformers faiss-cpu pandas tqdm


In [3]:
import sys

# remove conflicting packages
!{sys.executable} -m pip uninstall -y sentence-transformers transformers protobuf torch torchvision torchaudio

# install compatible versions (VERY IMPORTANT)
!{sys.executable} -m pip install torch==2.2.2
!{sys.executable} -m pip install transformers==4.41.2
!{sys.executable} -m pip install sentence-transformers==2.7.0
!{sys.executable} -m pip install python-pptx faiss-cpu pandas tqdm


Found existing installation: sentence-transformers 2.7.0
Uninstalling sentence-transformers-2.7.0:
  Successfully uninstalled sentence-transformers-2.7.0
Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: torch 2.2.2
Uninstalling torch-2.2.2:
  Successfully uninstalled torch-2.2.2
  Using cached torch-2.2.2-cp311-none-macosx_11_0_arm64.whl.metadata (25 kB)
Using cached torch-2.2.2-cp311-none-macosx_11_0_arm64.whl (59.7 MB)
  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
  Using cached sentence_transformers-2.7.0-py3-none-any.whl.metadata (11 kB)
Using cached sentence_transformers-2.7.0-py3-none-any.whl (171 kB)


In [25]:
import fitz   # PyMuPDF
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [26]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model loaded successfully 🚀")


Model loaded successfully 🚀


In [39]:
PDF_FOLDER = "ppt-folder"

In [40]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""

    for page in doc:
        text += page.get_text()

    return text


In [41]:
def chunk_text(text, chunk_size=200):
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)

    return chunks


In [42]:
def extract_metadata_from_filename(filename):
    name = filename.lower()

    semester = "unknown"
    subject = "unknown"

    # semester detect (sem-3, sem3 etc)
    for i in range(1,9):
        if f"sem-{i}" in name or f"sem{i}" in name:
            semester = f"Sem {i}"

    # subject detect (customize if needed)
    if "agile" in name:
        subject = "Agile Software Development"
    elif "ml" in name:
        subject = "Machine Learning"
    elif "dbms" in name:
        subject = "DBMS"
    elif "ai" in name:
        subject = "Artificial Intelligence"

    return semester, subject


In [43]:
all_chunks = []
all_metadata = []

files = [f for f in os.listdir(PDF_FOLDER) if f.endswith(".pdf")]

print("PDFs found:", files)

for file in tqdm(files):
    path = os.path.join(PDF_FOLDER, file)

    semester, subject = extract_metadata_from_filename(file)

    text = extract_text_from_pdf(path)
    chunks = chunk_text(text)

    for chunk in chunks:
        all_chunks.append(chunk)
        all_metadata.append({
            "semester": semester,
            "subject": subject,
            "file": file,
            "text": chunk
        })

print("Total chunks:", len(all_chunks))


PDFs found: ['CS_sem-3_agile_Module_5.pptx.pdf', 'CS_sem-3_agile_Module_4.pptx.pdf', 'CS_sem-3_agile_Module_2.pptx.pdf', 'CS_sem-3_agile_Module_3.pptx.pdf', 'CS3403_sem-6_NetworkSecurity_Module_1.pptx.pdf', 'CS3403_sem-6_NetworkSecurity_Module_4.pptx.pdf', 'CS3403_sem-6_NetworkSecurity_Module_5.pptx.pdf', 'CS3403_sem-6_NetworkSecurity_Module_3.pptx.pdf', 'CS3403_sem-6_NetworkSecurity_Module_2.pptx.pdf', 'CS_sem-3_agile_Module_1.pptx.pdf']


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  9.47it/s]

Total chunks: 315


In [44]:
embeddings = model.encode(
    all_chunks,
    batch_size=32,
    show_progress_bar=True
)

embeddings = np.array(embeddings).astype("float32")
print("Embeddings created:", embeddings.shape)


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Embeddings created: (315, 384)


In [45]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print("Stored vectors:", index.ntotal)


Stored vectors: 315


In [46]:
metadata_df = pd.DataFrame(all_metadata)

faiss.write_index(index, "university_db.index")
metadata_df.to_pickle("metadata.pkl")

print("Saved successfully 🚀")


Saved successfully 🚀


In [50]:
query = "What is 3 WAY HANDSHAKE?"

query_embedding = model.encode([query]).astype("float32")
D, I = index.search(query_embedding, k=5)

for idx in I[0]:
    print("\nTEXT:", metadata_df.iloc[idx]["text"])
    print("Semester:", metadata_df.iloc[idx]["semester"])
    print("Subject:", metadata_df.iloc[idx]["subject"])
    print("File:", metadata_df.iloc[idx]["file"])



TEXT: with the SYN bit field turned ON in the TCP header) specifying the port number of the peer to which it wants to connect and the client’s Initial Sequence Number is (x) Network Security (CS3403)– RVU – Mouli Sankaran 61 TCP Connection Establishment: 2 and 3 2. The server responds with its own SYN segment containing its initial sequence number (ISN(y)). This is the segment 2. The server also acknowledges the client’s SYN by ACK’ing ISN(x) plus 1. Note that a SYN consumes one sequence number and is retransmitted if lost. 3. The client acknowledges SYN from the server by ACK’ing ISN(y) plus 1. This is the segment 3. ⚫These three segments complete the connection establishment. ⚫This is often called the three-way handshake. ⚫Its main purposes are to let each end of the connection know that a connection is starting and the special details that may be carried as options, and to exchange the ISNs of both ends. Segment 2 Segment 3 Network Security (CS3403)– RVU – Mouli Sankaran 62 About S